In [1]:
import Donors_Choose_Prepro

In [7]:
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV,RepeatedKFold
from sklearn.utils import shuffle

In [3]:
X,y,features = Donors_Choose_Prepro.Pre_Pro(kaggle=False)

Importing Datasets
Precessing Resources & Merging Datasets
Date & Prefix Preprocessing


/Users/Nahel/GIT/Donors_Choose/Donors_Choose_Prepro.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  train.teacher_prefix[train.teacher_prefix.isnull()] = 'Teacher'


Encoding Categorical Features


HBox(children=(IntProgress(value=0, max=6), HTML(value='')))


Scaling Numeric Features
Text Processing
Essay Conversion & Scrub


HBox(children=(IntProgress(value=0, max=4), HTML(value='')))


Tf-idf processing


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))


Assigning X & y


In [9]:
feature_names = features

In [ ]:
# Build the model
cnt = 0
n_splits = 5
n_repeats = 1
kf = RepeatedKFold(
    n_splits=n_splits, 
    n_repeats=n_repeats, 
    random_state=42)

for train_index, valid_index in kf.split(X):
    print('Fold {}/{}'.format(cnt + 1, n_splits))
    
    params = {'is_unbalance': True, 
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.025,
        'bagging_freq': 5,
        'verbose': 1,
        'min_gain_to_split': 3,
        'min_child_samples' : 5,

        'num_threads': 1,
        'lambda_l2': 0.08,              
        'bagging_fraction': 0.85,
        'feature_fraction': 0.85,
        'max_depth': 18,
        'num_leaves': 24,
    }  

    lgbm_train = lgbm.Dataset(
        X.loc[train_index], 
        y.loc[train_index], 
        feature_name=feature_names)

    lgbm_valid = lgbm.Dataset(
        X.loc[valid_index], 
        y.loc[valid_index])

    model = lgbm.train(
        params, lgbm_train,
        num_boost_round=500,
        valid_sets=[lgbm_train, lgbm_valid],
        early_stopping_rounds=10,
        verbose_eval=10,)

    if cnt == 0:
        importance = model.feature_importance()
        model_fnames = model.feature_name()
        tuples = sorted(zip(model_fnames, importance), key=lambda x: x[1])[::-1]
        tuples = [x for x in tuples if x[1] > 0]
        print('Important features:')
        for i in range(60):
            if i < len(tuples):
                print(tuples[i])
            else:
                break
    
    cnt = cnt+1

Fold 1/5


In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
lgbm.plot_importance(model, max_num_features=15, height=0.8, ax=ax)
ax.grid(False)
plt.title("LightGBM - Feature Importance", fontsize=15)
plt.show()